In [1]:
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mlp
import random
from pathlib import Path
import os
from operator import itemgetter

import tensorflow.experimental.numpy as tnp
import numpy as np
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy import asarray
from numpy.random import randint
from numpy import savez_compressed
from numpy import load 
from numpy.random import randn

import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import backend
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Lambda
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import Reshape
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import load_img 
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Dropout 
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model
from tensorflow.keras.initializers import RandomNormal

# Load .npz files

In [2]:
# #### combining the npz
npz_path = "E:/UCF_Crimes/npz_120/"
npz_list = os.listdir(npz_path)

In [3]:
npz_list

['crimes_120_0.npz',
 'crimes_120_1.npz',
 'crimes_120_10.npz',
 'crimes_120_11.npz',
 'crimes_120_12.npz',
 'crimes_120_2.npz',
 'crimes_120_3.npz',
 'crimes_120_4.npz',
 'crimes_120_5.npz',
 'crimes_120_6.npz',
 'crimes_120_7.npz',
 'crimes_120_8.npz',
 'crimes_120_9.npz',
 'crimes_1_25570.npz',
 'crimes_1_25571.npz']

In [4]:
full_npz = itemgetter(0,1,3,4)(npz_list)

In [5]:
full_npz = [npz_path+npz for npz in full_npz]
full_npz

['E:/UCF_Crimes/npz_120/crimes_120_0.npz',
 'E:/UCF_Crimes/npz_120/crimes_120_1.npz',
 'E:/UCF_Crimes/npz_120/crimes_120_11.npz',
 'E:/UCF_Crimes/npz_120/crimes_120_12.npz']

In [6]:
data_all = [np.load(fname) for fname in full_npz] 

In [7]:
images = list()
images += [npz['arr_0'] for npz in data_all]

In [8]:
labels = list()
labels += [npz['arr_1'] for npz in data_all]

## AC-GAN discriminator model

In [9]:
# define the standalone discriminator model
def define_discriminator(in_shape=(120,120,3), n_classes=14):
    # weight initialization
    init = RandomNormal(stddev=0.02)
    # image input
    in_image = Input(shape=in_shape)
    
    # downsample to 60*60
    fe = Conv2D(16, (3,3), strides=(2,2), padding="same", kernel_initializer=init)(in_image)
    fe = LeakyReLU(alpha=0.2)(fe)
    fe = Dropout(0.5)(fe)
    # normal
    fe = Conv2D(32, (3,3), padding="same", kernel_initializer=init)(fe)
    fe = BatchNormalization()(fe)
    fe = LeakyReLU(alpha=0.2)(fe)
    fe = Dropout(0.5)(fe)
    
    # downsample to 30*30
    fe = Conv2D(64, (3,3), strides=(2,2), padding="same", kernel_initializer=init)(fe)
    fe = BatchNormalization()(fe)
    fe = LeakyReLU(alpha=0.2)(fe)
    fe = Dropout(0.5)(fe)
    # normal
    fe = Conv2D(128, (3,3), padding="same", kernel_initializer=init)(fe)
    fe = BatchNormalization()(fe)
    fe = LeakyReLU(alpha=0.2)(fe)
    
    fe = Dropout(0.5)(fe)

    # flatten feature maps
    fe = Flatten()(fe)
    # real/fake output
    out1 = Dense(1, activation="sigmoid")(fe)
    # class label output
    out2 = Dense(n_classes, activation="softmax")(fe)
    # define model
    model = Model(in_image, [out1, out2])
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss=["binary_crossentropy", "sparse_categorical_crossentropy"], optimizer=opt)
    return model


## Generator model

In [10]:
# define the standalone generator model
def define_generator(latent_dim, n_classes=14):
    # weight initialization
    init = RandomNormal(stddev=0.02)
    # label input
    in_label = Input(shape=(1,))
    # embedding for categorical input
    li = Embedding(n_classes, 50, input_length = 14)(in_label)
    
    n_nodes = 15 * 15
    li = Dense(n_nodes, kernel_initializer=init)(li)
    # reshape to additional channel
    li = Reshape((15, 15, 1))(li)
    # image generator input
    in_lat = Input(shape=(latent_dim,))
    
    
    # foundation for 15*15 image
    n_nodes = 128 * 15 * 15
    gen = Dense(n_nodes, kernel_initializer=init)(in_lat)
    gen = Activation("relu")(gen)
    gen = Reshape((15, 15, 128))(gen)
    # merge image gen and label input
    merge = Concatenate()([gen, li])
    # upsample to 30*30
    gen = Conv2DTranspose(32, (5,5), strides=(2,2), padding="same",kernel_initializer=init)(merge)
    gen = BatchNormalization()(gen)
    gen = Activation("relu")(gen)
    # upsample to 60*60    
    gen = Conv2DTranspose(64, (5,5), strides=(2,2), padding="same",kernel_initializer=init)(gen)
    gen = BatchNormalization()(gen)
    gen = Activation("relu")(gen)
    #upsample to 120*120
    gen = Conv2DTranspose(3, (5,5), strides=(2,2), padding="same", kernel_initializer=init)(gen)
    out_layer = Activation("tanh")(gen)
    # define model
    model = Model([in_lat, in_label], out_layer)
    return model


## Define GAN

In [11]:
# define the combined generator and discriminator model, for updating the generator
def define_gan(g_model, d_model):
# make weights in the discriminator not trainable
    for layer in d_model.layers:
        if not isinstance(layer, BatchNormalization):
            layer.trainable = False
    # connect the outputs of the generator to the inputs of the discriminator
    gan_output = d_model(g_model.output)
    # define gan model as taking noise and label and outputting real/fake and label outputs
    model = Model(g_model.input, gan_output)
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss=["binary_crossentropy", "sparse_categorical_crossentropy"], optimizer=opt)
    return model

## Select real samples

In [12]:
def generate_real_samples(x, y, n_samples, n_classes=14):
# split into images and labels    
    X_list, y_list = list(), list()
    n_per_class = int(n_samples / n_classes)
    for n in range (0, 4):
        for i in range(n_classes):
            # get all images for this class
            X_with_class = [x[n]][[y[n]] == i]
            # choose random instances
            ix = randint(0, len(X_with_class), n_per_class)
            # add to list
            [X_list.append(X_with_class[j]) for j in ix]
            [y_list.append(i) for j in ix]
    
    y = ones(((len(X_list)), 1))
#     ix = randint(0, x[0].shape[0], n_samples)
# # select images and labels
#     for n in range(0, 4): 
#         for i in x[n][ix]:
#             im_list.append(i)
#         for c in y[n][ix]:
#             lab_list.append(c)
# # generate class labels
#     y = ones(((n_samples*4), 1))
#     return asarray(X_list), asarray(y_list)
    return [asarray(X_list), asarray(y_list)], y

## Generate latent points

In [13]:
# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples, n_classes=14):
    # generate points in the latent space
    x_input = randn(latent_dim * n_samples)
    # reshape into a batch of inputs for the network
    z_input = x_input.reshape(n_samples, latent_dim)
    # generate labels
    labels = randint(0, n_classes, n_samples)
    return [z_input, labels]

## Generate fake examples

In [14]:
# use the generator to generate n fake examples, with class labels
def generate_fake_samples(generator, latent_dim, n_samples):
    # generate points in latent space
    z_input, labels_input = generate_latent_points(latent_dim, n_samples)
    # predict outputs
    images = generator.predict([z_input, labels_input])
    # create class labels
    y = zeros((n_samples, 1))
    return [images, labels_input], y

## summarize performance

In [15]:
def summarize_performance(step, g_model, latent_dim, n_samples=100):
# prepare fake examples
    [X, _], _ = generate_fake_samples(g_model, latent_dim, n_samples)
    # scale from [-1,1] to [0,1]
    X = (X + 1) / 2.0
    # plot images
    for i in range(9):
        # define subplot
        plt.subplot(3, 3, 1 + i)
        # turn off axis
        plt.axis("off")
        # plot raw pixel data
        plt.imshow(X[i, :, :, 0])
    # save plot to file
    filename1 = "generated_plot_%04d.png" % (step+1)
    pyplot.savefig(filename1)
    pyplot.close()
    
#     # save the generator model
#     filename2 = "model_g_%04d.h5" % (step+1)
#     g_model.save(filename2)
    #save the discriminator model
    filename3 = "model_d_%04d.h5"% (step+1)
    d_model.save(filename3)
    print(">Saved: %s and %s" % (filename1,filename3))

## Train model

In [16]:
# train the generator and discriminator
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=100, n_batch=20):
    # calculate the number of batches per training epoch
    bat_per_epo = int(dataset[0][0].shape[0] / n_batch)
    # calculate the number of training iterations
    n_steps = bat_per_epo * n_epochs
    # calculate the size of half a batch of samples
    half_batch = int(n_batch* 5)
    print('n_epochs=%d, n_batch=%d, 1/2=%d, bat_per_e=%d, steps=%d' % (n_epochs, n_batch, half_batch, bat_per_epo, n_steps))
    
    [X_real, labels_real], y_real = generate_real_samples(images, labels, half_batch)
    print(X_real.shape, labels_real.shape, y_real.shape)
    
    # manually enumerate epochs
    for i in range(n_steps):
        # get randomly selected ✬real✬ samples
        [X_real, labels_real], y_real = generate_real_samples(images, labels, half_batch)
        
        # update discriminator model weights
        _, d_r1, d_r2 = d_model.train_on_batch(X_real, [y_real, labels_real])
        
        # generate ✬fake✬ examples
        [X_fake, labels_fake], y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
        
        # update discriminator model weights
        _,d_f, d_f2 = d_model.train_on_batch(X_fake, [y_fake, labels_fake])
        
        # prepare points in latent space as input for the generator
        [z_input, z_labels] = generate_latent_points(latent_dim, n_batch)
        
        # create inverted labels for the fake samples
        y_gan = ones((n_batch, 1))
        
        # update the generator via the discriminator✬s error
        _,g_1,g_2 = gan_model.train_on_batch([z_input, z_labels], [y_gan, z_labels])
        
        # summarize loss on this batch
        print(">%d, dr[%.3f,%.3f], df[%.3f,%.3f], g[%.3f,%.3f]" % (i+1, d_r1, d_r2, d_f, d_f2, g_1,g_2))
        
        # evaluate the model performance every ✬epoch✬
        if (i+1) % (bat_per_epo * 10) == 0:
            summarize_performance(i, g_model, latent_dim)

## Latent space

In [17]:
# size of the latent space
latent_dim = 100

## Discriminator

In [18]:
discriminator = define_discriminator()

## Generator

In [19]:
generator = define_generator(latent_dim)

## GAN

In [20]:
gan_model = define_gan(generator, discriminator)

## Dataset

In [21]:
dataset = images, labels

## train model

In [ ]:
train(generator, discriminator, gan_model, dataset, latent_dim)

n_epochs=100, n_batch=20, 1/2=100, bat_per_e=499, steps=49900
(392, 120, 120, 3) (392,) (392, 1)
>1, dr[1.012,3.479], df[3.204,3.197], g[0.015,4.152]
>2, dr[0.020,4.195], df[1.472,3.212], g[0.566,4.639]
>3, dr[0.259,5.743], df[0.778,2.836], g[0.923,6.250]
>4, dr[0.273,6.509], df[0.808,2.740], g[0.276,5.307]
>5, dr[0.076,6.959], df[0.831,2.920], g[0.287,8.077]
>6, dr[0.103,8.614], df[0.779,2.778], g[0.669,7.935]
>7, dr[0.090,8.714], df[0.765,2.699], g[0.298,7.761]
>8, dr[0.093,7.776], df[0.781,2.825], g[0.495,7.809]
>9, dr[0.079,8.003], df[0.774,2.844], g[0.354,9.617]
>10, dr[0.058,8.819], df[0.743,2.726], g[0.341,9.472]
>11, dr[0.052,8.739], df[0.747,2.716], g[0.479,11.419]
>12, dr[0.077,8.662], df[0.736,2.829], g[0.429,8.755]
>13, dr[0.062,7.192], df[0.734,2.801], g[0.410,8.079]
>14, dr[0.068,8.203], df[0.733,2.734], g[0.292,7.035]
>15, dr[0.062,7.248], df[0.727,2.691], g[0.171,6.341]
>16, dr[0.051,6.266], df[0.729,2.700], g[0.364,6.302]
>17, dr[0.076,6.674], df[0.722,2.704], g[0.174,

>151, dr[0.002,3.803], df[0.005,2.824], g[0.051,3.450]
>152, dr[0.002,3.769], df[0.013,2.654], g[0.027,3.692]
>153, dr[0.002,3.694], df[0.010,2.703], g[0.062,3.747]
>154, dr[0.002,3.714], df[0.009,2.681], g[0.033,3.330]
>155, dr[0.002,3.640], df[0.006,2.659], g[0.070,3.704]
>156, dr[0.002,3.853], df[0.004,2.778], g[0.040,3.512]
>157, dr[0.002,3.713], df[0.006,2.715], g[0.028,3.166]
>158, dr[0.002,3.736], df[0.006,2.773], g[0.013,3.341]
>159, dr[0.002,3.915], df[0.008,2.662], g[0.018,3.188]
>160, dr[0.001,3.714], df[0.008,2.743], g[0.060,3.434]
>161, dr[0.002,3.832], df[0.007,2.691], g[0.010,3.156]
>162, dr[0.002,3.848], df[0.006,2.766], g[0.023,3.280]
>163, dr[0.003,3.812], df[0.004,2.723], g[0.032,3.417]
>164, dr[0.003,3.882], df[0.003,2.795], g[0.032,3.326]
>165, dr[0.002,3.872], df[0.008,2.674], g[0.010,3.751]
>166, dr[0.003,3.807], df[0.007,2.770], g[0.012,3.843]
>167, dr[0.002,3.846], df[0.008,2.764], g[0.019,3.032]
>168, dr[0.001,3.734], df[0.006,2.723], g[0.032,3.516]
>169, dr[0

>300, dr[0.005,2.997], df[0.002,2.743], g[0.097,2.951]
>301, dr[0.002,2.926], df[0.002,2.880], g[0.034,2.863]
>302, dr[0.003,2.850], df[0.003,2.739], g[0.009,3.083]
>303, dr[0.003,2.916], df[0.003,2.725], g[0.014,2.797]
>304, dr[0.004,2.893], df[0.001,2.796], g[0.024,2.937]
>305, dr[0.003,2.898], df[0.001,2.845], g[0.006,3.142]
>306, dr[0.002,2.938], df[0.001,2.832], g[0.102,3.128]
>307, dr[0.002,2.955], df[0.002,2.810], g[0.041,3.092]
>308, dr[0.003,2.937], df[0.002,2.732], g[0.019,3.263]
>309, dr[0.002,2.949], df[0.005,2.796], g[0.020,2.742]
>310, dr[0.004,2.893], df[0.003,2.799], g[0.016,2.947]
>311, dr[0.002,2.943], df[0.003,2.726], g[0.009,3.362]
>312, dr[0.003,2.955], df[0.002,2.794], g[0.039,3.085]
>313, dr[0.003,2.950], df[0.002,2.854], g[0.060,3.214]
>314, dr[0.003,2.993], df[0.002,2.703], g[0.041,3.008]
>315, dr[0.002,3.011], df[0.002,2.693], g[0.035,2.753]
>316, dr[0.003,2.929], df[0.002,2.816], g[0.020,2.803]
>317, dr[0.003,2.913], df[0.005,2.805], g[0.007,3.173]
>318, dr[0

>448, dr[2.737,3.586], df[0.000,4.268], g[7.825,3.591]
>449, dr[0.687,3.585], df[1.548,3.713], g[21.984,3.094]
>450, dr[2.183,3.620], df[0.000,3.556], g[17.320,3.904]
>451, dr[1.477,3.558], df[0.001,3.505], g[2.329,3.722]
>452, dr[0.079,3.565], df[1.060,3.696], g[10.454,4.096]
>453, dr[1.086,3.769], df[0.000,3.498], g[7.366,4.220]
>454, dr[0.580,3.547], df[0.037,3.644], g[1.021,3.274]
>455, dr[0.121,3.956], df[0.707,3.823], g[3.890,4.521]
>456, dr[0.436,3.874], df[0.043,3.749], g[2.006,3.620]
>457, dr[0.493,3.570], df[1.059,3.703], g[6.709,4.409]
>458, dr[1.979,3.500], df[1.215,3.846], g[7.934,4.073]
>459, dr[1.571,3.744], df[0.443,4.156], g[4.653,4.948]
>460, dr[0.408,3.717], df[0.211,4.523], g[4.786,2.858]
>461, dr[0.318,3.519], df[0.152,4.093], g[4.064,4.278]
>462, dr[0.194,3.546], df[0.136,4.107], g[4.077,3.795]
>463, dr[0.146,3.505], df[0.063,3.331], g[3.511,3.555]
>464, dr[0.117,3.438], df[0.176,3.558], g[3.035,4.033]
>465, dr[0.203,3.209], df[0.118,3.440], g[2.050,3.727]
>466, d

>597, dr[0.054,2.964], df[0.037,2.977], g[0.237,3.202]
>598, dr[0.058,3.044], df[0.035,2.759], g[0.215,3.117]
>599, dr[0.028,3.019], df[0.045,2.812], g[0.141,3.029]
>600, dr[0.042,2.955], df[0.047,2.843], g[0.261,2.911]
>601, dr[0.028,3.015], df[0.030,2.700], g[0.483,3.182]
>602, dr[0.047,3.009], df[0.023,2.995], g[0.093,3.086]
>603, dr[0.019,2.919], df[0.050,2.916], g[0.208,3.150]
>604, dr[0.051,2.926], df[0.056,2.861], g[0.217,3.022]
>605, dr[0.049,3.031], df[0.013,2.925], g[0.082,2.722]
>606, dr[0.014,3.007], df[0.081,2.885], g[0.261,3.047]
>607, dr[0.041,3.066], df[0.016,2.846], g[0.167,3.314]
>608, dr[0.025,3.011], df[0.030,2.952], g[0.315,2.990]
>609, dr[0.024,2.932], df[0.039,2.855], g[0.303,3.081]
>610, dr[0.028,3.028], df[0.030,2.819], g[0.130,3.080]
>611, dr[0.030,2.953], df[0.030,2.901], g[0.094,3.152]
>612, dr[0.025,2.944], df[0.017,2.851], g[0.074,2.755]
>613, dr[0.019,3.008], df[0.077,2.906], g[0.368,3.132]
>614, dr[0.127,3.044], df[0.120,2.969], g[0.285,3.182]
>615, dr[0

...

>1632, dr[0.020,2.991], df[0.012,0.053], g[0.104,0.075]
>1633, dr[0.013,2.972], df[0.012,0.056], g[0.018,0.098]
>1634, dr[0.016,2.958], df[0.020,0.055], g[0.058,0.062]
>1635, dr[0.012,2.980], df[0.027,0.074], g[0.084,0.055]
>1636, dr[0.023,3.012], df[0.016,0.065], g[0.032,0.089]
>1637, dr[0.017,2.950], df[0.010,0.061], g[0.072,0.030]
>1638, dr[0.020,2.899], df[0.016,0.035], g[0.045,0.032]
>1639, dr[0.016,2.950], df[0.014,0.064], g[0.053,0.034]
>1640, dr[0.021,2.870], df[0.042,0.066], g[0.266,0.032]
>1641, dr[0.022,2.897], df[0.022,0.049], g[0.097,0.034]
>1642, dr[0.031,2.937], df[0.053,0.049], g[0.103,0.088]
>1643, dr[0.041,2.960], df[0.057,0.071], g[0.138,0.055]
>1644, dr[0.048,2.988], df[0.053,0.056], g[0.119,0.059]
>1645, dr[0.073,2.978], df[0.164,0.084], g[0.301,0.075]
>1646, dr[0.229,2.913], df[0.503,0.055], g[0.795,0.085]
>1647, dr[0.512,2.993], df[1.094,0.069], g[3.832,0.043]
>1648, dr[1.619,3.041], df[1.113,0.125], g[1.912,0.170]
>1649, dr[0.258,2.954], df[0.102,0.097], g[3.398

>1779, dr[0.046,3.138], df[0.049,0.044], g[0.209,0.108]
>1780, dr[0.049,3.150], df[0.058,0.050], g[0.236,0.132]
>1781, dr[0.050,3.037], df[0.034,0.042], g[0.121,0.088]
>1782, dr[0.073,3.024], df[0.078,0.042], g[0.162,0.058]
>1783, dr[0.077,3.019], df[0.062,0.038], g[0.158,0.053]
>1784, dr[0.054,3.042], df[0.076,0.041], g[0.074,0.222]
>1785, dr[0.061,3.008], df[0.049,0.043], g[0.158,0.116]
>1786, dr[0.099,3.000], df[0.065,0.031], g[0.100,0.102]
>1787, dr[0.035,3.114], df[0.036,0.043], g[0.148,0.094]
>1788, dr[0.077,3.002], df[0.025,0.034], g[0.211,0.121]
>1789, dr[0.033,3.039], df[0.072,0.037], g[0.232,0.047]
>1790, dr[0.063,2.959], df[0.032,0.052], g[0.171,0.139]
>1791, dr[0.053,2.988], df[0.049,0.045], g[0.046,0.108]
>1792, dr[0.021,3.054], df[0.057,0.043], g[0.174,0.090]
>1793, dr[0.069,3.008], df[0.031,0.045], g[0.114,0.110]
>1794, dr[0.062,2.995], df[0.040,0.037], g[0.090,0.095]
>1795, dr[0.045,2.973], df[0.041,0.033], g[0.175,0.033]
>1796, dr[0.032,2.941], df[0.102,0.036], g[0.375

>1926, dr[0.098,3.130], df[0.167,0.117], g[1.042,0.421]
>1927, dr[0.259,3.073], df[0.241,0.100], g[1.266,0.229]
>1928, dr[0.152,3.042], df[0.139,0.101], g[2.157,0.141]
>1929, dr[0.170,3.123], df[0.485,0.137], g[2.493,0.718]
>1930, dr[1.037,3.067], df[1.943,0.321], g[7.525,0.306]
>1931, dr[1.682,3.088], df[0.232,0.077], g[2.974,0.442]
>1932, dr[0.066,3.324], df[0.463,0.056], g[5.465,0.213]
>1933, dr[0.358,3.155], df[0.103,0.033], g[4.112,0.279]
>1934, dr[0.259,3.103], df[0.268,0.037], g[3.438,0.170]
>1935, dr[0.127,3.017], df[0.230,0.045], g[4.314,0.250]
>1936, dr[0.158,3.094], df[0.086,0.020], g[4.319,0.116]
>1937, dr[0.105,3.035], df[0.194,0.022], g[3.261,0.287]
>1938, dr[0.093,2.968], df[0.288,0.051], g[4.142,0.103]
>1939, dr[0.158,2.900], df[0.023,0.044], g[2.643,0.181]
>1940, dr[0.075,3.003], df[0.081,0.051], g[1.137,0.263]
>1941, dr[0.053,2.973], df[0.038,0.073], g[0.556,0.435]
>1942, dr[0.115,3.042], df[0.087,0.065], g[0.880,0.223]
>1943, dr[0.076,2.981], df[0.041,0.069], g[0.788

>2073, dr[0.308,3.139], df[0.308,0.079], g[3.323,0.113]
>2074, dr[0.335,3.050], df[0.323,0.070], g[2.352,0.165]
>2075, dr[0.302,3.075], df[0.245,0.077], g[2.463,0.108]
>2076, dr[0.282,3.167], df[0.347,0.107], g[3.106,0.174]
>2077, dr[0.372,3.094], df[0.288,0.066], g[2.692,0.126]
>2078, dr[0.239,3.132], df[0.242,0.061], g[2.689,0.175]
>2079, dr[0.252,3.114], df[0.149,0.054], g[1.624,0.115]
>2080, dr[0.132,3.068], df[0.316,0.087], g[2.825,0.415]
>2081, dr[0.339,3.006], df[0.253,0.075], g[2.424,0.170]
>2082, dr[0.347,2.979], df[0.275,0.112], g[2.294,0.249]
>2083, dr[0.269,3.004], df[0.598,0.081], g[4.919,0.170]
>2084, dr[0.806,3.061], df[0.309,0.211], g[2.262,0.284]
>2085, dr[0.277,3.047], df[0.679,0.149], g[5.038,0.236]
>2086, dr[0.783,3.061], df[0.307,0.165], g[3.238,0.110]
>2087, dr[0.331,3.048], df[0.590,0.109], g[3.823,0.168]
>2088, dr[0.519,3.054], df[0.347,0.097], g[3.923,0.264]
>2089, dr[0.351,3.089], df[0.333,0.122], g[3.913,0.236]
>2090, dr[0.417,3.106], df[0.264,0.119], g[3.523

>2220, dr[0.122,3.062], df[0.061,0.133], g[0.499,0.077]
>2221, dr[0.084,3.044], df[0.082,0.174], g[0.294,0.206]
>2222, dr[0.072,3.129], df[0.076,0.125], g[0.540,0.298]
>2223, dr[0.100,3.124], df[0.188,0.103], g[0.690,0.245]
>2224, dr[0.143,3.081], df[0.094,0.072], g[0.736,0.168]
>2225, dr[0.128,3.037], df[0.156,0.070], g[1.134,0.193]
>2226, dr[0.120,3.151], df[0.131,0.071], g[0.679,0.218]
>2227, dr[0.170,3.040], df[0.221,0.059], g[0.690,0.283]
>2228, dr[0.275,3.083], df[0.169,0.063], g[1.137,0.202]
>2229, dr[0.145,3.048], df[0.374,0.079], g[3.365,0.252]
>2230, dr[0.906,2.984], df[1.962,0.036], g[3.864,0.174]
>2231, dr[1.512,2.969], df[0.870,0.150], g[2.054,0.269]
>2232, dr[0.140,3.064], df[0.079,0.107], g[2.919,0.261]
>2233, dr[0.291,3.022], df[0.443,0.059], g[2.319,0.106]
>2234, dr[0.339,3.030], df[0.248,0.097], g[1.721,0.172]
>2235, dr[0.189,3.128], df[0.254,0.108], g[1.310,0.176]
>2236, dr[0.159,3.135], df[0.188,0.091], g[1.826,0.169]
>2237, dr[0.167,3.063], df[0.179,0.069], g[1.189